In [1]:
# Preamble 
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import astropy.units as u
from astropy.table import Table, join

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import corner
from matplotlib.colors import LogNorm

try:
    from zero_point import zpt
    zpt.load_tables()
except:
    raise ImportError('You need to install the package zero_point from here: https://gitlab.com/icc-ub/public/gaiadr3_zeropoint')


In [2]:
setup = 'allstar'
version = 240207

dr4_data = Table.read('galah_dr4_'+setup+'_'+str(version)+'.fits')
dr4_data = dr4_data[['sobject_id']]

## Cross-match of GALAH via 2MASS ID to Gaia DR3 and then via source_id to WISE & BailerJones
```
SELECT
galah.sobject_id,
tmass.designation as tmass_id,
tmass.ph_qual as tmass_ph_qual,
tmass.ra as raj2000,
tmass.dec as dej2000,
tmass.j_m, tmass.j_msigcom,
tmass.h_m, tmass.h_msigcom,
tmass.ks_m, tmass.ks_msigcom,
allwise.designation as wise_id,
allwise.w1mpro as W1mag,
allwise.w1mpro_error as e_W1mag,
allwise.w2mpro as W2mag,
allwise.w2mpro_error as e_W2mag,
allwise.w3mpro as W3mag,
allwise.w3mpro_error as e_W3mag,
allwise.w4mpro as W4mag,
allwise.w4mpro_error as e_W4mag,
allwise.ph_qual as wise_ph_qual,
calj.r_med_geo,calj.r_lo_geo,calj.r_hi_geo,
calj.r_med_photogeo,calj.r_lo_photogeo,calj.r_hi_photogeo,
calj.flag as calj_flag,
gaia.*,
galah.ebv
FROM gaiadr3.gaia_source as gaia
LEFT OUTER JOIN
	external.gaiaedr3_distance as calj
	ON calj.source_id = gaia.source_id
LEFT OUTER JOIN
	gaiadr3.allwise_best_neighbour AS wisexmatch
	ON wisexmatch.source_id = gaia.source_id
LEFT OUTER JOIN
	gaiadr1.allwise_original_valid as allwise
	ON allwise.allwise_oid = wisexmatch.allwise_oid
LEFT OUTER JOIN
	gaiadr3.tmass_psc_xsc_best_neighbour AS tmassxmatch
	ON tmassxmatch.source_id = gaia.source_id
LEFT OUTER JOIN
	gaiadr3.tmass_psc_xsc_join AS tmass_join
	ON tmass_join.clean_tmass_psc_xsc_oid = tmassxmatch.clean_tmass_psc_xsc_oid
LEFT OUTER JOIN
	gaiadr1.tmass_original_valid as tmass
	ON tmass.designation = tmass_join.original_psc_source_id
INNER JOIN
	user_sbuder.dr6_0_230718_unique_ids as galah
	ON galah.tmass_id = tmass.designation
```

In [3]:
tmass_gaiadr3 = Table.read('../auxiliary_information/dr60_231004_tmass_gaiadr3_xmatch.fits')

In [4]:
# # The correction of Gaia Gband magnitudes is already included in Gaia DR3.
# # So no need to use this function from https://github.com/agabrown/gaiaedr3-6p-gband-correction
# # Prepare the photometric information from Gaia eDR3
# gmag_corr, gflux_corr = correct_gband(
#     bp_rp = tmass_gaiadr3['bp_rp'], 
#     astrometric_params_solved = tmass_gaiadr3['astrometric_params_solved'], 
#     phot_g_mean_mag = tmass_gaiadr3['phot_g_mean_mag'], 
#     phot_g_mean_flux = tmass_gaiadr3['phot_g_mean_flux']
# )
# tmass_gaiadr3['phot_g_mean_mag'] = gmag_corr
# tmass_gaiadr3['phot_g_mean_flux'] = gflux_corr

# Calculation of Gaia eDR3 G/GBP/GRP uncertainties:
# See both https://www.cosmos.esa.int/web/gaia/edr3-passbands
# and https://cdsarc.unistra.fr/viz-bin/ReadMe/I/350?format=html&tex=true#sRM3.63
sigmaG_0 = 0.0027553202
sigmaGBP_0 = 0.0027901700
sigmaGRP_0 = 0.0037793818
tmass_gaiadr3['phot_g_mean_mag_error']   = np.sqrt((-2.5/np.log(10)*tmass_gaiadr3['phot_g_mean_flux_error']/tmass_gaiadr3['phot_g_mean_flux'])**2 + sigmaG_0**2)
tmass_gaiadr3['phot_bp_mean_mag_error'] = np.sqrt((-2.5/np.log(10)*tmass_gaiadr3['phot_bp_mean_flux_error']/tmass_gaiadr3['phot_bp_mean_flux'])**2 + sigmaGBP_0**2)
tmass_gaiadr3['phot_rp_mean_mag_error'] = np.sqrt((-2.5/np.log(10)*tmass_gaiadr3['phot_rp_mean_flux_error']/tmass_gaiadr3['phot_rp_mean_flux'])**2 + sigmaGRP_0**2)

# Calculations of Parallax Zeropoint Corrections:
# Following script by Lindegren et al. (2021b)
# https://ui.adsabs.harvard.edu/abs/2021A&A...649A...4L
tmass_gaiadr3['plx_zpt_corr'] = np.zeros(len(tmass_gaiadr3['source_id']))
has_astrometric_params_solved_31_95 = (tmass_gaiadr3['astrometric_params_solved'] == 31) | (tmass_gaiadr3['astrometric_params_solved'] == 95)

# Parallax zeropoint corretion for 
tmass_gaiadr3['plx_zpt_corr'][has_astrometric_params_solved_31_95] = zpt.get_zpt(
    phot_g_mean_mag = tmass_gaiadr3['phot_g_mean_mag'][has_astrometric_params_solved_31_95],
    nu_eff_used_in_astrometry = tmass_gaiadr3['nu_eff_used_in_astrometry'][has_astrometric_params_solved_31_95],
    pseudocolour = tmass_gaiadr3['pseudocolour'][has_astrometric_params_solved_31_95],
    ecl_lat = tmass_gaiadr3['ecl_lat'][has_astrometric_params_solved_31_95],
    astrometric_params_solved = tmass_gaiadr3['astrometric_params_solved'][has_astrometric_params_solved_31_95]
)

tmass_gaiadr3['parallax_raw'] = tmass_gaiadr3['parallax']
tmass_gaiadr3['parallax_error_raw'] = tmass_gaiadr3['parallax_error']
tmass_gaiadr3['parallax'] = tmass_gaiadr3['parallax'] - tmass_gaiadr3['plx_zpt_corr']

/Users/buder/opt/anaconda3/lib/python3.9/site-packages/zero_point/zpt.py:205: UserWarning: The apparent magnitude of one or more of the sources is outside the expected range (6-21 mag). 
                Outside this range, there is no further interpolation, thus the values at 6 or 21 are returned.
  warnings.warn(
/Users/buder/opt/anaconda3/lib/python3.9/site-packages/zero_point/zpt.py:220: UserWarning: The nu_eff_used_in_astrometry of some of the 5p source(s) is outside the expected range (1.1-1.9 
                mag). Outside this range, the zero-point calculated can be seriously wrong.
  warnings.warn(
/Users/buder/opt/anaconda3/lib/python3.9/site-packages/zero_point/zpt.py:233: UserWarning: The pseudocolour of some of the 6p source(s) is outside the expected range (1.24-1.72 mag).
                 The maximum corrections are reached already at 1.24 and 1.72
  warnings.warn(


In [5]:
# There are possibly multiple matches for the same sobject_id. We only take the brightest one in Gaia DR3's Gmag
tmass_gaiadr3.sort(keys='phot_g_mean_mag')
unique_sobject_ids, unique_sobject_id_indices, inverse = np.unique(tmass_gaiadr3['sobject_id'], return_index=True, return_inverse=True)
tmass_gaiadr3 = tmass_gaiadr3[unique_sobject_id_indices]

In [6]:
tmass_gaiadr3 = tmass_gaiadr3[(~tmass_gaiadr3['tmass_id'].mask)]

In [7]:
dr4_tmass_gaiadr3 = join(dr4_data, tmass_gaiadr3, keys='sobject_id', join_type='left')

In [8]:
dr4_tmass_gaiadr3.write('galah_dr4_vac_wise_tmass_gaiadr3_240207.fits',overwrite=True)